In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
from src.leaflet_creation import create_map_visualization, get_color_for_df

In [3]:
true_pred = pd.read_csv('poster used true and pred.csv',index_col=0)

In [4]:
seg = gpd.read_file('data/segments_dc.geojson')

In [5]:
%matplotlib inline

In [6]:
simplified = seg[['geometry']].merge(true_pred, left_index=True, right_index=True, how='right')

In [17]:
true = simplified[['geometry', 'csl']].reset_index()
pred = simplified[['geometry', 'pred']].reset_index()

In [18]:
def getColor(d):
    if d > 4: return '#1a9641'  # green (4, ]
    if d > 3: return '#a6d96a'  # light green (3, 4]
    if d > 2: return '#ffffbf'  # yellow: (2, 3]
    if d > 1: return '#fdae61'  # orange: (1, 2]
    return '#d7191c'            # red: (, 1]


In [19]:
true['color'] = true.csl.apply(getColor)
pred['color'] = pred.pred.apply(getColor)

In [20]:
html_title = 'poster map'
file_path = ''
file_name = 'poster map'
lon, lat = -77.0908494, 38.9045525  #D.C.
zoom = 12
init_layers = ['streets', 'stsg']
map_layers = ['light','streets', 'satellite']
binding_data=[['true_map','actual map'],['pred_map','predicted map']]
# gpdf1['color'] = '#aa0'
# gpdf2['color'] = '#0a0'
gpdfs = [true, pred]
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs, width=1000)